Copyright (c) Microsoft Corporation.<br>
Licensed under the MIT License.

# 1. Collect Data from the Azure Percept DK Vision

In this notebook we will:
- Learn how to connect to cameras on the dev kit and collect data

**Prerequisites to run the notebooks if have not completed already**

Follow the `readme.md` for information on setting up the Percept DK for this notebook.

In [ ]:
import os
import threading
import time

import cv2
import matplotlib.pyplot as plt

import ipywidgets
from IPython.display import display
import traitlets

from camera.capture_camera import Camera
from datasets.dataset import ImageDataset

## Helper methods

In [ ]:
# A callback function to update the image in image widget
def update_image(change):
    """Update an image within a traitlets image widget"""
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)

In [ ]:
def detach_camera(camera):
    """Method to detach a camera."""
    try:
        camera.running = False
        camera.release()
    except Exception as err:
        print("Error with camera object.  Did you specify the RTSP URL correctly?")
    try:
        camera.unobserve(update_image, names='value')
    except ValueError as err:
        print("Error calling 'unobserve'.")

In [ ]:
def bgr8_to_jpeg(frame):
    """Convert from BGR int8 to jpg format"""
    return bytes(cv2.imencode('.jpg', frame)[1])

## Initialize the camera(s)

You must run this to attach your Vision camera (Eye) to the notebook for data collection.  Replace the `<ip-address-of-percept>` with the IP of your Percept DK as you found in the setup of the device (also address you use for SSH).  If the camera object is already associated with a camera it detaches it and the cell must be run again to reassociate it. 

In [ ]:
try:
    # Camera object from CaptureCamera class
    camera1 = Camera(width=816,
                     height=616,
                     capture_device="rtsp://<ip-address-of-percept>:8554/rawTCP")


    # Set camera to running
    camera1.running = True
    print("Camera is now associated with the notebook.")
except Exception as err:
    print("Camera not created.  Error: {}".format(err))
    print("Detaching camera as specified capture_device location.",
          "Try running cell again.")
    try:
        detach_camera(camera1)
    except NameError as err:
        print('Camera object is not defined.  Did you make sure to:\n\
               1. On the device, add TCP rule with iptables:  sudo iptables -A INPUT -p tcp --dport 8554 -j ACCEPT\n\
               2. Add this in the deployment manifest port bindings for Vision Eye module and redeploy modules to device: "8554/tcp":[{"HostPort":"8554"}]')

## Set up for data collection

Let's define some variables for setting up data collection.  Update the following as needed to set the label(s) for your object(s).

In [ ]:
# Update this as needed - this is also the name of a folder that
# gets created.
TASK = 'office_items'

# Update this as needed - generally just need one "bucket" category
# and labeling comes later in Azure ML
CATEGORIES = ['office_supplies']

# Update this as needed - these are the same object, but different 
# datasets in case there is need for this like restarting the
# experiments with fresh data
DATASETS = ['A','B']

# --------------------------- NO NEED TO MODIFY THE REST WITHIN THIS CELL ---------------------------

DATA_DIR = os.path.join(os.getcwd(),'data/')

datasets = {}
for name in DATASETS:
    datasets[name] = ImageDataset(DATA_DIR + os.sep + TASK + '_' + name, 
                                  CATEGORIES)

print("{} task with {} categories defined".format(TASK, CATEGORIES))

In [ ]:
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
    print('Created data directory: {}'.format(DATA_DIR))

## Create data collection widget

Below we create a data collection widget with `ipywidgets` and `traitlets` and prepare to use the dataset we set up above.

In [ ]:
# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
camera1.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image()
traitlets.dlink((camera1, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')
save_widget = ipywidgets.Button(description='add')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

# save image for category and update counts
def save(c):
    dataset.save_entry(camera1.value, category_widget.value)
    count_widget.value = dataset.get_count(category_widget.value)
save_widget.on_click(save)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget]), dataset_widget, category_widget, count_widget, save_widget
])

print("data_collection_widget created")

## Show widget

View the entire data collection UI.  Use the `add` button to add as many images as you like - there is a counter.  The images are collected locally to your machine in the `data` folder.

Plan to collect 150-200 images per class for a decent model.  You may start with ~30-50 to understand the process.  If this is multiclass, collect an even number of images for each class.  Vary the position, lighting, etc. for a more robust model or use data augmentation later on.

In [ ]:
# Combine into one display
all_widget = ipywidgets.VBox([
    ipywidgets.HBox([data_collection_widget])
])

display(all_widget)

## Release cameras _only_ when done with data collection

We are now done with data collection and can detach the camera.

In [ ]:
# Will need if camera is still live or running
# If camera will not shut down, use the last line in this cell
camera1.running = False
try:
    camera1.release()
    camera1.unobserve(update_image, names='value')
except Exception as err:
    print("Error calling 'unobserve' likely because 'value' not in names.")
    print('Error: {}'.format(err))
    
# Optionally you may simply stop the Python kernel - uncomment below
# os._exit(00)

## Next steps

- Upload the data to Azure ML Workspace and label it for object detection